In [1]:
import mturk
import random
from datetime import datetime
import json
from pprint import pprint
import copy
import numpy as np
from collections import OrderedDict
import pandas as pd
import pymongo
import botocore
import uuid
import pdb
import seaborn as sns
import matplotlib.pyplot as plt
import pingouin as pg
import krippendorff_alpha as ka

C:\Users\gabri\anaconda3\envs\phd\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.0.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
data_folder = './data/campaign/batch_2/'
config_folder = './config/campaign/batch_1_and_2/'

with open('./data/language_tests.json','r',encoding='utf-8') as f:
    language_tests = json.load(f)
with open('./config/mongodb_credentials.json','r') as f:
    mongodb_credentials = json.load(f)
    #mongodb_credentials["connection_string"]

In [3]:
""" Connect to MTurk and to the Mongodb database. Set the boolean below to TRUE to use the marketplace and to FALSE to use the sandbox (testing the HITs)"""
create_hits_in_production = False
is_pilot = False

db_client = pymongo.MongoClient(mongodb_credentials["connection_string"])
db = db_client['verbalisations']

collection_name = 'hit_results' + ('_sandbox' if not create_hits_in_production else '') + ('_pilots' if is_pilot else '')

hit_result_collection = db[collection_name]
hit_result_collection

Collection(Database(MongoClient(host=['cluster0-shard-00-00.hjstc.mongodb.net:27017', 'cluster0-shard-00-02.hjstc.mongodb.net:27017', 'cluster0-shard-00-01.hjstc.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='Cluster0-shard-0', ssl=True), 'verbalisations'), 'hit_results_sandbox')

In [4]:
mt = mturk.MTurk()
mt.launch_client(create_hits_in_production)
collection_name

10000.00


'hit_results_sandbox'

In [5]:
""" Ban the spammers! """
to_ban = False
if to_ban:
    with open('./config/banlist.json','r') as f:
        banlist = json.load(f)
    for w in banlist:
        try:
            response = mt.client.create_worker_block(
                WorkerId=w,
                Reason='You are copy and pasting text'
            )
            assert(response['ResponseMetadata']['HTTPStatusCode'] == 200)
        except botocore.exceptions.ClientError as e:
            continue

In [27]:
""" Create the tasks by populating the HTML templates using the config file """

task_types = ['fluency','adequacy']
task_type = task_types[1]

with open(config_folder + 'task_config_{}.json'.format(task_type),'r') as f:
    task_content = json.load(f)

TaskAttributes = task_content['task_attributes']

with open(task_content['html_layout'], 'r', encoding='utf-8') as f:
    html_layout = f.read()
    

html_layout = html_layout.replace('${time_thr}$', task_content['time_thr'])
if is_pilot:
    html_layout = html_layout.replace('${pilot_wording}$',
    '''
    <li>
        <span style="color:red">This is a pilot.</span> Helpful and constructive feedback, regardless of
        whether you finished the task, will be compensated. If there are technical errors that prevented
        you from finishing, let us know and we will take them into consideration.
    </li>
    '''                              
    )
else:
    html_layout = html_layout.replace('${pilot_wording}$','')
    
with open(data_folder + task_content['tasks'],'r') as f:
    taskSets = json.load(f)
    
# If you're only testing, just pick one hit and run it once, with no qualification barriers
if not create_hits_in_production:
    TaskAttributes.pop('QualificationRequirements')
    TaskAttributes['MaxAssignments'] = 1 
    random.seed(42)
    taskSets = random.sample(taskSets,1)
        
taskSets_dict = {
    'html_layout' : html_layout,
    'taskSets' : taskSets,
    'TaskAttributes' : TaskAttributes,
    'task_content': task_content
}
      
print(f'Generated {len(taskSets)} tasks with the following configs:')

pprint(TaskAttributes,indent=1) #verify the properties before running the HITs

Generated 1 tasks with the following configs:
{'AssignmentDurationInSeconds': 10800,
 'Description': 'Help us by measuring the adequacy of some sentences compared '
                'to background information. You should have reading '
                'proficiency in English. Constructive feedback will be '
                'appreciated and possibly rewarded.',
 'Keywords': 'Adequacy, English, Reading, Comparing information, Comprehension',
 'LifetimeInSeconds': 604800,
 'MaxAssignments': 1,
 'Reward': '1',
 'Title': 'Measuring the adequacy of English sentences'}


In [28]:
""" See how many HITs this will generate, already multiplied by the expected number of assignments.
Multiply the resulting number by the payment to see how much money this batch will consume. """
    
l=[]
done_count=0
target_assignments = TaskAttributes['MaxAssignments']
for taskSet in taskSets:
    TaskAttributes_hit = copy.deepcopy(TaskAttributes)
    TaskAttributes_hit['MaxAssignments'] = target_assignments -\
        sum([hit['hit']['NumberOfAssignmentsCompleted'] for hit in hit_result_collection.find({
            'taskSet_id':taskSet['_id'],
            'type': task_content['type'],
        })]) #or sum [hit['hit']['NumberOfAssignmentsCompleted'] for completed results
    l.append(TaskAttributes_hit['MaxAssignments'])
    if TaskAttributes_hit['MaxAssignments'] == 0:
        done_count += 1
print(f'Remaining HITs: {sum(l)}')
print(f'Tasks totally finished: {done_count}')

Remaining HITs: 1
Tasks totally finished: 0


In [29]:
""" Create the batch of HITs """

results = []
batch_id = str(uuid.uuid4())

#start_from = 0

hit_type_id = ''
target_assignments = TaskAttributes['MaxAssignments']
for idx, taskSet in enumerate(taskSets):
    print(f"{idx}: {taskSet['_id']}")
    #if idx<start_from:
    #    continue
    TaskAttributes_hit = copy.deepcopy(TaskAttributes) # Adjust based on how many were already done in other batches
    TaskAttributes_hit['MaxAssignments'] = target_assignments -\
        sum([hit['hit']['NumberOfAssignmentsCompleted'] for hit in hit_result_collection.find({
            'taskSet_id':taskSet['_id'],
            'type': task_content['type']
        })])
    if TaskAttributes_hit['MaxAssignments'] > 0:
        try:
            random.seed(None)
            language_questions = random.sample(language_tests['en'],k=4)
            response = mt.create_hit(
                html_layout.replace('${verbalisations}$', str(taskSet['taskSet'])).\
                            replace('${attention_questions}$', json.dumps(language_questions)),
                **TaskAttributes_hit
            )

            hit_type_id = response['HIT']['HITTypeId']
            result = {
                '_id': response['HIT']['HITId'],
                'batch_id': batch_id,
                'type': task_content['type'],
                'verbalisations': taskSet['taskSet'],
                'attention_test': language_questions,
                'taskSet_id':taskSet['_id'],
                'hit': response['HIT'],
                'timestamp': datetime.now()
            }
            results.append(result)
            hit_result_collection.insert_one(result)
        except botocore.exceptions.ClientError as e:
            print(e.__dict__)
            if e.response['Error']['Code'] == 'RequestError':
                # Not enough funds
                print("Funds ran out! The last hit above was not launched! Please recharge!")
                break
            elif e.response['Error']['Code'] == 'ThrottlingException':    
                pdb.set_trace()              
                print("Turn off the database updater!")
                continue
            else:
                pdb.set_trace()            
                raise
        except Exception as e:
            pdb.set_trace()            
            raise
        #except:
        #    pdb.set_trace()            
        #    raise

# For you to go to the HITs you just created and test them
print('Launched tasks')
if not create_hits_in_production:
    print('You can view the HITs here:')
    print(mt.mturk_environment['preview']+"?groupId={}".format(hit_type_id))
else:
    print('Launched! Good Luck!')
    
print('Batch ID is',batch_id)

0: 1d4adeff-958a-4226-9aab-2bf08931e37e
Launched tasks
You can view the HITs here:
https://workersandbox.mturk.com/mturk/preview?groupId=3H2ZGQZOT0LUSWYACA6AW95XXAGXYX
Batch ID is c09133f7-1de0-4491-b291-d743a140f335


In [30]:
# REMOVING FINISHED TASKS FROM THE UPDATE QUEUE

""" If you set 'force' to TRUE, it will abort mission and force an expiry in all HITs and then delete them.
If you only want to remove the completed ones (make them Disposed so the update routine won't loop through tons of
HITs), keep it as FALSE."""
force = True
while True:
    ''' Dispose all hits in the database '''
    query = {'hit.HITStatus': {'$not': {'$eq': 'Disposed'}}}
    #query['type'] = 'relevance'
    if not force:
        query['hit.NumberOfAssignmentsPending'] = 0
        query['hit.NumberOfAssignmentsAvailable'] = 0
    elif force:
        query['hit.NumberOfAssignmentsPending'] = 0
    hit_result_collection_list = list(hit_result_collection.find(query))
    if (not force and len(hit_result_collection_list) == 0) or (force and mt.client.list_hits()['NumResults']==0):
    #if len(list(hit_result_collection.find({'hit.HITStatus': {'$not': {'$eq': 'Disposed'}}, 'type':'relevance'}))) == 0:
        print('Finished')
        break
    for hit in hit_result_collection_list:
        try:
            mt.client.delete_hit(HITId = hit['_id'])
            print('Removed',hit['_id'])
        except Exception as e:
            #print('Level 1', hit['_id'], e)
            if force:
                try:
                    mt.client.update_expiration_for_hit(HITId = hit['_id'], ExpireAt=datetime(2017, 1, 1))
                    mt.client.delete_hit(HITId = hit['_id'])
                    print('Removed',hit['_id'])
                except Exception as e:
                    #print('Level 2', hit['_id'],e)
                    pass
            continue

Removed 3BAKUKE4AKOTF3TCXUNS9PF9B3J1RF
Finished


# MISC

In [36]:
mt.client.get_account_balance()

{'AvailableBalance': '10000.00',
 'ResponseMetadata': {'RequestId': 'cac83b9b-d08b-477e-a444-cb6347002975',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'cac83b9b-d08b-477e-a444-cb6347002975',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '31',
   'date': 'Wed, 06 Oct 2021 16:05:21 GMT'},
  'RetryAttempts': 0}}

In [8]:
#mt.client.send_bonus(
#    WorkerId='-',
#    BonusAmount='-',
#    AssignmentId='-',
#    Reason='feedback'
#)

{'ResponseMetadata': {'RequestId': 'a038b160-0721-4475-bbfd-ee73cf5fe3b9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a038b160-0721-4475-bbfd-ee73cf5fe3b9',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'date': 'Fri, 13 Aug 2021 18:45:26 GMT'},
  'RetryAttempts': 0}}

In [38]:
# CLEANING LEFTOVERS

mt.client.list_hits(MaxResults=100)
#hitid = '3M93N4X8INZCBZ9T28UOVALJBK2SJO'
#mt.client.update_expiration_for_hit(HITId = hitid, ExpireAt=datetime(2018, 1, 1))
#mt.get_hit_answers(HITId = hitid, approve=True)
#mt.client.delete_hit(HITId = hitid)

{'NumResults': 0,
 'HITs': [],
 'ResponseMetadata': {'RequestId': '473b5051-fa30-48cd-8b87-4bd024bfebdf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '473b5051-fa30-48cd-8b87-4bd024bfebdf',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '26',
   'date': 'Wed, 06 Oct 2021 16:07:04 GMT'},
  'RetryAttempts': 0}}